# 03 Deploy to Amazon Bedrock

## Amazon Bedrock Imported Models란?

생성형 AI 분야가 빠르게 발전함에 따라, 기존 모델보다 더 나은 가격 대비 성능을 제공하는 **Mistral, Llama**와 같이 개방형 아키텍처를 갖춘 새로운 모델이 계속 등장하고 있습니다. 고객은 특정 비즈니스 요구 사항을 해결하기 위해 이러한 모델을 특정 사용 사례에 맞게 파인 튜닝하는 경우가 많습니다. 

이전에는 Bedrock 외부에서 사용자 지정한 모델의 경우 자체 관리형 인프라에 배포해야 했기 때문에 애플리케이션 개발자가 서로 다른 모델 소스 간에 전환할 때 일관성 없는 경험 문제가 발생했습니다. 

Amazon Bedrock Imported Model 출시로 이제 지원되는 모델 아키텍처에 대해 Bedrock 외부에서 사용자 지정된 이러한 모델을 가져와서 Bedrock의 호출 모델 API를 통해 온디맨드로 액세스할 수 있으므로 **Amazon Bedrock에서 기본 모델, 사용자 지정 모델 및 가져온 모델 전반에 걸쳐 통합된 경험을 구축**할 수 있습니다.

#### 지원되는 모델
- Mistral - Sliding Window Attention (SWA)과 Grouped Query Attention (GQA)에 대한 옵션이 있는 디코더 전용 변환기 기반 아키텍처입니다.
- Flan — 인코더 디코더 기반 변환기 모델인 T5 아키텍처의 향상된 버전입니다. 
- Llama 2/Llama3 — Grouped Query Attention (GQA)에 대한 옵션이 있는 향상된 버전의 Llama 모델입니다.

#### 비용 체계
![pricing](../images/deploy_pricing.png)

> Llama 3.1 8B 128K 모델은 ‘소형’ 모델 사본 크기로 가져오고, Llama 3.1 70B 128K 모델은 ‘대형’ 모델 사본 크기로 가져옵니다.

- 사용자 지정 모델 가져오기를 사용하면 Bedrock의 기존 호스팅된 파운데이션 모델과 동일한 **완전관리형 방식**으로 사용할 수 있습니다. 지원되는 모델 아키텍처의 사용자 지정 가중치를 가져오고 온디맨드 모드를 사용하여 사용자 지정 모델을 제공할 수 있습니다. 
- 사용자 지정 모델을 Bedrock으로 가져오는 데는 비용이 들지 않습니다. 
- 모델을 가져오면 컨트롤 플레인 작업을 수행할 필요 없이 온디맨드로 모델에 액세스할 수 있습니다. 추론을 처리하는 데 필요한 사용자 지정 모델의 사본 수와 각 모델 사본이 활성화된 기간을 기준으로 모델 추론 비용만 청구되며, 요금은 **5분 단위로 청구**됩니다. 

- 모델 사본은 추론 요청을 처리할 준비가 된 가져온 모델의 단일 인스턴스입니다. 모델 사본의 분당 요금은 아키텍처, 컨텍스트 길이, AWS 리전, 컴퓨팅 유닛 버전(하드웨어 세대)과 같은 요소에 따라 달라지며 모델 사본 크기에 따라 계층화됩니다. 

#### 요금 예제
>애플리케이션 개발자가 us-east-1 리전에서 8B 파라미터 크기의 128K 시퀀스 길이를 가진 사용자 지정 Llama 3.1 유형 모델을 가져옵니다. 이는 위 표에 나와 있는 ‘소형’ 크기의 모델 사본에 해당하며, 분당 모델 사본당 단가는 0.1570 USD입니다.

모델을 가져오는 데는 요금이 부과되지 않습니다. 첫 번째 성공적인 간접 호출은 오전 8시 3분이며, 이때 측정이 시작됩니다. 5분 측정 기간은 오전 8시 3분부터 오전 8시 7분, 오전 8시 7분부터 오전 8시 11분 등입니다. 5분 기간 중 하나 이상의 간접 호출이 있는 경우 해당 기간은 청구 대상 기간으로 간주됩니다. 오전 8시 7분부터 오전 8시 11분까지 간접 호출이 없는 경우, 오전 8시 11분에 측정이 중지됩니다. 이 경우 청구서는 다음과 같이 계산됩니다. 0.1570 USD * 5 분 * 3건의 5분 기간 = 2.355 USD

## Amazon Bedrock Imported Models 사용하기

### Step 1 

![step1](../images/deploy_1.png)

- Amazon Bedrock 콘솔에서 `Imported models`를 클릭합니다.

### Step 2 

![step2](../images/deploy_2.png)

- `Imported model`을 클릭하여 배포 작업을 시작합니다.

### Step 3

![step3](../images/deploy_3.png)

- `Model name`을 입력하고, 파인 튜닝한 모델을 저장한 S3 location을 선택한 후 `Import model`을 클릭합니다.
- S3 경로는 `{S3버킷}/{training job 이름}/output/model/ `로 설정해야합니다.

### Step 4

![step4](../images/deploy_4.png)

- Model import job은 llama-8b 기준 약 15분이 소요됩니다.
- Model import job이 완료되면 `Models`에서 배포된 모델을 확인할 수 있습니다.

### Step 5

![step5](../images/deploy_5.png)

- 배포가 완료되면 `Bedrock Playground`에서 모델을 테스트해볼 수 있습니다. 
- 아래 sample question으로 시작해보세요.

    ```
    <s>[INST]<<SYS>>
    
    You are a powerful text-to-SQL model. Your job is to answer questions about a database. You can use the following table schema for context: CREATE TABLE table_name_11 (tournament VARCHAR)
    <</SYS>>

    [INST]
    Human:
    Return the SQL query that answers the following question: 1987년에 A가 있었던 대회는 어떤 대회입니까?

    [/INST]
    Assistant:
    ```

## Amazon Bedrock API로 응답 확인하기

- Amazon Bedrock의 invoke API로 llama3 모델을 불러옵니다.

In [ ]:
import boto3
import sagemaker
import json

sess = sagemaker.Session()

region = sess.boto_region_name
client = boto3.client("bedrock-runtime", region_name=region)
model_id = "<ENTER_YOUR_MODEL_ARN_HERE>"

assert model_id != "<ENTER_YOUR_MODEL_ARN_HERE>", "ERROR: Please enter your model id"

def get_sql_query(system_prompt, user_question):
    """
    Generate a SQL query using Llama 3 8B
    Remember to use the same template used in fine tuning
    """
    formatted_prompt = f"<s>[INST] <<SYS>>{system_prompt}<</SYS>>\n\n[INST]Human: Return the SQL query that answers the following question: {user_question}[/INST]\n\nAssistant:"
    native_request = {
        "prompt": formatted_prompt,
        "max_tokens": 100,
        "top_p": 0.9,
        "temperature": 0.1
    }
    response = client.invoke_model(modelId=model_id,
                                   body=json.dumps(native_request))
    response_text = json.loads(response.get('body').read())["outputs"][0]["text"]

    return response_text

### Test set에 있는 질문 실행해보기

In [ ]:
import json

# 데이터셋을 저장할 리스트
dataset = []

# JSON 파일에서 데이터 읽기 (이 부분은 실제 데이터 로딩 방식에 따라 조정 필요)
with open('../datasets/ko_test_dataset.json', 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        dataset.append(data['messages'])

# 무작위로 10개의 예제 선택 (실제 환경에서는 random.sample 등을 사용할 수 있습니다)
sample_dataset = dataset[:10]

# 각 데이터셋 항목에 대해 쿼리 생성 및 결과 비교
for item in sample_dataset:
    system_prompt = item[0]['content']
    user_question = item[1]['content'].split(": ", 1)[1]  # "Return the SQL query that answers the following question: " 부분 제거
    assistant_answer = item[2]['content']
    
    # 모델을 사용하여 SQL 쿼리 생성
    generated_query = get_sql_query(system_prompt, user_question)
    
    print(f"질문: {user_question}\n")
    print(f"답변: {generated_query}\n")
    print(f"정답: {assistant_answer}\n")
    print("\n" + "="*50 + "\n")

- `user_question`에 한국어 질문을 직접 작성하여 실행해보세요.

In [ ]:
system_prompt = "You are a powerful text-to-SQL model. Your job is to answer questions about a database. You can use the following table schema for context: CREATE TABLE table_name_11 (tournament VARCHAR)"
user_question = "1987년에 A가 있었던 대회는 어떤 대회입니까?"

query = get_sql_query(system_prompt, user_question).strip()
print(query)